In [2]:
df = spark.read.csv('file:/home/hduser/Titanic.csv',inferSchema=True, header=True)

In [5]:
rm_columns = df.select(['Survived','Pclass', 
                       'Sex','Age','SibSp', 
                       'Parch','Fare','Embarked'])
result = rm_columns.na.drop() 

In [7]:
# Importing the required libraries 
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder 
  
# Converting the Sex Column 
sexIdx = StringIndexer(inputCol='Sex', 
                               outputCol='SexIndex') 
sexEncode = OneHotEncoder(inputCol='SexIndex', 
                               outputCol='SexVec') 
  
# Converting the Embarked Column 
embarkIdx = StringIndexer(inputCol='Embarked', 
                               outputCol='EmbarkIndex') 
embarkEncode = OneHotEncoder(inputCol='EmbarkIndex', 
                               outputCol='EmbarkVec') 
  
# Vectorizing the data into a new column "features"  
# which will be our input/features class 
assembler = VectorAssembler(inputCols=['Pclass', 
                                       'SexVec','Age', 
                                       'SibSp','Parch', 
                                       'Fare','EmbarkVec'], 
                                    outputCol='features')

In [13]:
# Importing Pipeline and Model 
from pyspark.ml import Pipeline 
from pyspark.ml.classification import LogisticRegression 
  
log_reg = LogisticRegression(featuresCol='features', 
                             labelCol='Survived') 
  
# Creating the pipeline 
pipeline = Pipeline(stages=[sexIdx, embarkIdx, 
                            sexEncode, embarkEncode, 
                            assembler, log_reg]) 

#After pipelining the tasks, we will split the data into training data and testing data to train and test the model.

# Splitting the data into train and test 
train_data, test_data = result.randomSplit([0.7, .3]) 
  
# Fitting the model on training data 
fit_model = pipeline.fit(train_data) 
  
# Storing the results on test data 
results = fit_model.transform(test_data) 


# Showing the results 
results.show() 

+--------+------+----+----+-----+-----+--------+--------+--------+-----------+-------------+-------------+--------------------+--------------------+--------------------+----------+
|Survived|Pclass| Sex| Age|SibSp|Parch|    Fare|Embarked|SexIndex|EmbarkIndex|       SexVec|    EmbarkVec|            features|       rawPrediction|         probability|prediction|
+--------+------+----+----+-----+-----+--------+--------+--------+-----------+-------------+-------------+--------------------+--------------------+--------------------+----------+
|       0|     1|male|13.0|    2|    2| 262.375|       C|     0.0|        1.0|(1,[0],[1.0])|(2,[1],[1.0])|[1.0,1.0,13.0,2.0...|[22.7984337834299...|[0.99999999987446...|       0.0|
|       0|     1|male|17.0|    0|    0|    47.1|       S|     0.0|        0.0|(1,[0],[1.0])|(2,[0],[1.0])|[1.0,1.0,17.0,0.0...|[29.4072689801657...|[0.99999999999983...|       0.0|
|       0|     1|male|23.0|    0|    0|    93.5|       S|     0.0|        0.0|(1,[0],[1.0])|(2,

In [16]:
#Model evaluation using ROC-AUC:

# Importing the evaluator 
from pyspark.ml.evaluation import BinaryClassificationEvaluator 
  
# Calling the evaluator 
res = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived') 
  
# Evaluating the AUC on results 
ROC_AUC = res.evaluate(results) 

print(ROC_AUC)
#In general, an AUC value above 0.7 is considered good


1.0


In [3]:
test_data1 = test_data.drop("Survived")
results = fit_model.transform(test_data1) 
results.show()

NameError: name 'test_data' is not defined

In [2]:
df = spark.read.csv('file:/home/hduser/Titanic_realdata.csv',inferSchema=True, header=True)
df.show()

+-----------+--------+------+--------------------+----+----+-----+-----+------------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name| Sex| Age|SibSp|Parch|            Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+----+----+-----+-----+------------------+-------+-----+--------+
|       1934|       1|     3|Goldsmith, Mr. Na...|male|  32|    0|    0|SOTON/O.Q. 3101263|   7.85| null|       S|
|       1935|       1|     2|Corbett, Mrs. Wal...|male|  30|    0|    0|            237249|   13.0| null|       S|
|       1936|       1|     1|Kimball, Mrs. Edw...|male|  45|    1|    0|             11753|52.5542|  D19|       S|
|       1937|       1|     3|Peltomaki, Mr. Ni...|male|  25|    0|    0| STON/O 2. 3101291|  7.925| null|       S|
|       1938|       1|     1|Chevre, Mr. Paul ...|male|  45|    0|    0|          PC 17594|   29.7|   A9|       C|
|       1939|       1|     3|Shaughnessy, Mr. ...|male|null|    0|    0|        